# Scrapear Amazon con Selenium y Scrapy

In [1]:
import re
from scrapy import Request
from scrapy.selector import Selector
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--headless")  # Enable headless mode

## Url a procesar

In [3]:
url='https://www.amazon.ae/Duracell-1-45V-Zinc-Hearing-Batteries/dp/B0BJFH3YYL/ref=sr_1_1?crid=3GKZLLF8NTSUL&keywords=B0BJFH3YYL&qid=1707493734&sprefix=b08njlhgdh%2Caps%2C256&sr=8-1'

In [4]:
driver = webdriver.Chrome(options=chrome_options)

In [5]:
driver.get(url)

In [6]:
html = driver.page_source

In [7]:
driver.quit()

In [8]:
selector = Selector(text=html)
response = selector

In [9]:
print(selector)

<Selector query=None data='<html lang="en-ae" class=" a-js a-aud...'>


## Extraccion de la informacion

Titulo

In [10]:
title = selector.css('#productTitle::text').get()
print(title)

        Duracell | 1.45V Zinc Air Hearing Aid Batteries Size 675 Pack of 6       


Precio

In [11]:
price = selector.css('#newAccordionRow_0 > div:nth-child(1) > div:nth-child(1) > h5:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1) > span:nth-child(1)::text').get()
print(price)

None


Best Sellers Rank

In [12]:
category_best_sellers_rank_container = selector.css("#productDetails_detailBullets_sections1 > tbody > tr > td > span")
best_sellers_rank_element = category_best_sellers_rank_container.css("span span")
print(len(best_sellers_rank_element))

0


In [13]:
for i in best_sellers_rank_element:
    print(i)

In [14]:
best_sellers_rank_content = []
for i in best_sellers_rank_element:
    pattern = r">([^<]+)<"
    matches = re.findall(pattern, i.extract(), re.DOTALL)
    cadena = "".join(matches)
    best_sellers_rank_content.append(cadena)

In [15]:
print(best_sellers_rank_content)

[]


In [16]:
contenedor = selector.css("#productDetails_detailBullets_sections1 > td > span")
print(contenedor)
print(len(contenedor))

[]
0


In [17]:
span = contenedor.css('span span').getall()
print(span)

[]


In [18]:
list = []
for i in span:
    if i != ' ' and i != '  ' and i != '':
        print(f"{i}\n")
        list.append(i)
        

In [19]:
list_clean = []
for i in list:
    pattern = r">([^<]+)<"
    matches = re.findall(pattern, i, re.DOTALL)
    cadena = "".join(matches)
    list_clean.append(cadena)

In [20]:
print(list_clean)

[]


# Capturar otros provedores del producto

In [21]:
others_sellers = selector.css('.mbcMerchantName::text').get()

In [22]:
print(others_sellers)

 Sea wonders (Free Installation for Gym Equipments) 


In [23]:
others_offer = selector.css('div.a-section.a-spacing-none span.a-size-small.mbcMerchantName::text').getall()

In [24]:
print(others_offer)

[' Sea wonders (Free Installation for Gym Equipments) ', ' propell ']


In [25]:
for i in others_offer:
    print(i)

 Sea wonders (Free Installation for Gym Equipments) 
 propell 


In [26]:
brand_store_href = selector.css('#bylineInfo::attr(href)').get()
brand_store_link = f"https://www.amazon.ae{brand_store_href}"

In [27]:
print(brand_store_link)

https://www.amazon.ae/stores/Duracell/page/8126018F-4B33-4E27-8353-534BE8B54BA5?ref_=ast_bln


In [28]:
others_offer_container = selector.css('div.a-box-inner.a-padding-small span.a-declarative div.a-row')                                    
print(others_offer_container)

[<Selector query="descendant-or-self::div[(@class and contains(concat(' ', normalize-space(@class), ' '), ' a-box-inner ')) and (@class and contains(concat(' ', normalize-space(@class), ' '), ' a-padding-small '))]/descendant-or-self::*/span[@class and contains(concat(' ', normalize-space(@class), ' '), ' a-declarative ')]/descendant-or-self::*/div[@class and contains(concat(' ', normalize-space(@class), ' '), ' a-row ')]" data='<div class="a-row">              <spa...'>, <Selector query="descendant-or-self::div[(@class and contains(concat(' ', normalize-space(@class), ' '), ' a-box-inner ')) and (@class and contains(concat(' ', normalize-space(@class), ' '), ' a-padding-small '))]/descendant-or-self::*/span[@class and contains(concat(' ', normalize-space(@class), ' '), ' a-declarative ')]/descendant-or-self::*/div[@class and contains(concat(' ', normalize-space(@class), ' '), ' a-row ')]" data='<div class="a-row">              <spa...'>]


In [29]:
others_price = selector.css('div.a-box-inner.a-padding-small span.a-declarative div.a-row span.a-size-medium.a-color-price::text').getall()
print(others_price)

[' AED\xa025.00 ', ' AED\xa020.00 ']


In [30]:
others_offer_container = selector.css('div.a-box-inner.a-padding-small span.a-declarative div.a-row')         
print(f"OTHERS OFFER:{others_offer}\n")
for i in others_offer_container:
    title_store = i.css('div.a-section.a-spacing-none span.a-size-small.mbcMerchantName::text').get()
    print(title_store)
    if 'Amazon.ae' in title_store:
        final_price = i.css('span.a-size-medium.a-color-price::text').get()
        is_transmed = True
        print(f"Encontre un Strore Transmed")

OTHERS OFFER:[' Sea wonders (Free Installation for Gym Equipments) ', ' propell ']

 Sea wonders (Free Installation for Gym Equipments) 
 propell 


In [31]:
#Capturo el elemento que tiene el contenido
best_sellers_rank_container = response.css('#productDetails_detailBullets_sections1 > tr:nth-child(3) > td:nth-child(2) > span:nth-child(1)')
#Si el elemento no tiene Customer Reviews cambia el selector
if best_sellers_rank_container == []:
    best_sellers_rank_container = response.css('#productDetails_detailBullets_sections1 > tr:nth-child(2) > td:nth-child(2) > span:nth-child(1)')
if best_sellers_rank_container == None or best_sellers_rank_container == []:
    best_sellers_rank_container = response.css('#productDetails_detailBullets_sections1 > tr > td > span')
    
try:
    if best_sellers_rank_container != None:
        #Extraigo el contenido como lista de elementos
        best_sellers_rank_element = best_sellers_rank_container.css("span span")
        
        #Extraigo solamente el contenido de cada elemento
        best_sellers_rank_content = []
        if best_sellers_rank_element:
            for i in best_sellers_rank_element:
                pattern = r">([^<]+)<"
                matches = re.findall(pattern, i.extract(), re.DOTALL)
                cadena = "".join(matches)
                best_sellers_rank_content.append(cadena)
            
            #print(f"El elemento best_sellers_rank_container:{best_sellers_rank_content}")# Print para testear el elemento  
            
        if len(best_sellers_rank_content) == 3:
            catgroup_best_sellers_rank = best_sellers_rank_content[0]
            # category_best_sellers_rank = f"{best_sellers_rank_content[1]}, {best_sellers_rank_content[2]}"
            category_best_sellers_rank = f"{best_sellers_rank_content[1]}"
        else:
            catgroup_best_sellers_rank = best_sellers_rank_content[0]
            category_best_sellers_rank = best_sellers_rank_content[1]
    #Elimino (See Top 100 in Grocery) de la cadena
    category_best_sellers_rank = re.sub(r"[^\d]", "", category_best_sellers_rank)
    catgroup_best_sellers_rank = re.sub(r"\(.*?\)", "", catgroup_best_sellers_rank)
    catgroup_best_sellers_rank = re.sub(r"[^\d]", "", catgroup_best_sellers_rank)

except:
    catgroup_best_sellers_rank = ""
    category_best_sellers_rank = ""
    print(f"ADVERTENCIA No se pudo extraer Best Selle Rank")            
            

ADVERTENCIA No se pudo extraer Best Selle Rank


In [32]:
print(catgroup_best_sellers_rank)
print(category_best_sellers_rank)

In [99]:
if best_sellers_rank_container == None or best_sellers_rank_container == []:
    best_sellers_string = ''
    best_sellers_list = response.css('ul.a-unordered-list:nth-child(4) > li:nth-child(1) > span:nth-child(1) ::text').getall()
    best_sellers_string = "".join(best_sellers_list)
    best_sellers_string = re.findall(r"#(\d+)", best_sellers_string)
    catgroup_best_sellers_rank = best_sellers_string[0]
    category_best_sellers_rank = best_sellers_string[1]     

In [100]:
print(catgroup_best_sellers_rank)
print(category_best_sellers_rank)

61
931
